# Machine learning:
1. Imagine you have a dataset where you have different Instagram features like u sername , Caption , Hashtag , Followers , Time_Since_posted , and likes , now your task is to predict the number of likes and Time Since posted and the rest of the features are your input features. Now you have to build a model which can predict the number of likes and Time Since posted. 
Dataset This is the Dataset You can use this dataset for this question.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import warnings

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler
from sklearn.model_selection import  train_test_split, GridSearchCV, cross_validate, cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor

from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
df = pd.read_csv('C:\Assessment\Ineuron_assessment\Machine leraning\instagram_reach.csv',na_values="na")
df.head()

,Unnamed: 0,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
0,0,1,mikequindazzi,Who are #DataScientist and what do they do? >>...,1600,#MachineLearning #AI #DataAnalytics #DataScien...,11 hours,139
1,1,2,drgorillapaints,We all know where it’s going. We just have to ...,880,#deck .#mac #macintosh#sayhello #apple #steve...,2 hours,23
2,2,3,aitrading_official,Alexander Barinov: 4 years as CFO in multinati...,255,#whoiswho #aitrading #ai #aitradingteam#instat...,2 hours,25
3,3,4,opensourcedworkplace,sfad,340,#iot #cre#workplace #CDO #bigdata #technology#...,3 hours,49
4,4,5,crea.vision,Ever missed a call while your phone was chargi...,304,#instamachinelearning #instabigdata#instamarke...,3 hours,30


In [7]:
df.shape

(100, 8)

In [3]:
# df info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         100 non-null    int64 
 1   S.No               100 non-null    int64 
 2   USERNAME           100 non-null    object
 3   Caption            94 non-null     object
 4   Followers          100 non-null    int64 
 5   Hashtags           100 non-null    object
 6   Time since posted  100 non-null    object
 7   Likes              100 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 6.4+ KB


In [6]:
df['Unnamed: 0'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26], dtype=int64)

In [3]:
df.columns

Index(['Unnamed: 0', 'S.No', 'USERNAME', 'Caption', 'Followers', 'Hashtags',
       'Time since posted', 'Likes'],
      dtype='object')

In [4]:
df.shape

(100, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         100 non-null    int64 
 1   S.No               100 non-null    int64 
 2   USERNAME           100 non-null    object
 3   Caption            94 non-null     object
 4   Followers          100 non-null    int64 
 5   Hashtags           100 non-null    object
 6   Time since posted  100 non-null    object
 7   Likes              100 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 6.4+ KB


In [12]:
df.duplicated().sum()

0

In [10]:
df['Hashtags'].unique()

array(['#MachineLearning #AI #DataAnalytics #DataScienc #DataLake',
       '\xa0#deck\xa0.#mac\xa0#macintosh#sayhello\xa0#apple\xa0#stevejobs\xa0#ai\xa0#evolution#artificialintelligence\xa0#machinelearning#terminator\xa0#illbeback\xa0#technology#computerevolution\xa0#computerscience#sciencefiction#computersciencetosciencefiction#tomorrowstechnology\xa0#vr\xa0#ar\xa0#robot#robots\xa0#t2\xa0#businessdeck\xa0#businessslides#illustration\xa0#sketches\xa0#drawing',
       '#whoiswho\xa0#aitrading\xa0#ai\xa0#aitradingteam#instateam\xa0#instapeople\xa0#ai\xa0#trading#artificialintelligence\xa0#crypto#cryptocurrency\xa0#blockchain\xa0#tradingforex#forex\xa0#fiatmoney\xa0#coins\xa0#machinelearning#userexperience\xa0#instamachinelearning#instabigdata\xa0#instamarketing#artificialintelligence\xa0#deeplearning#datascience\xa0#industry\xa0#marketing#bigdata\xa0#datascience\xa0#machinelearning#ml',
       '#iot\xa0#cre#workplace\xa0#CDO\xa0#bigdata\xa0#technology#tech\xa0#work\xa0#computerscience\xa

In [13]:
# Dropping these columns since these columns dont have any relation to the output feature.
df = df.drop(['Unnamed: 0', 'S.No', 'USERNAME','Caption', 'Hashtags'], axis=1)

In [18]:
df.Followers.nunique()

94

In [14]:
df.head()

,Followers,Time since posted,Likes
0,1600,11 hours,139
1,880,2 hours,23
2,255,2 hours,25
3,340,3 hours,49
4,304,3 hours,30


In [15]:
df.columns

Index(['Followers', 'Time since posted', 'Likes'], dtype='object')

In [16]:
# define numerical & categorical columns
numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 2 numerical features : ['Followers', 'Likes']

We have 1 categorical features : ['Time since posted']


In [19]:
# removing hours and typecasting to int
df['Time since posted'] = df['Time since posted'].map(lambda a: int(re.sub('hours', '', a)))


In [20]:
df.dtypes

Followers            int64
Time since posted    int64
Likes                int64
dtype: object

In [21]:
df.head()

,Followers,Time since posted,Likes
0,1600,11,139
1,880,2,23
2,255,2,25
3,340,3,49
4,304,3,30


In [22]:
# Splitting X and y for all Experiments
X = df.drop(['Likes', 'Time since posted'], axis=1)
y = df[['Likes', 'Time since posted']]

In [23]:
X.head()

,Followers
0,1600
1,880
2,255
3,340
4,304


In [24]:
y.head()

,Likes,Time since posted
0,139,11
1,23,2
2,25,2
3,49,3
4,30,3


In [25]:
def evaluate_clf(true, predicted):
    '''
    This function takes in true values and predicted values
    Returns: MAE, MSE, RMSE, r2score
    '''
    MAE = mean_absolute_error(true, predicted) # Calculate Mean Absolute Error
    MSE = mean_squared_error(true, predicted) # Calculate Mean Squared Error
    RMSE =np.sqrt(MSE) # Calculate Root Mean Squared Error
    r2score = r2_score(true, predicted)  # Calculate R_squaredScore/R2score
    return MAE, MSE, RMSE, r2score

In [26]:
# Dictionary which contains models for experiment
models = {
    "Linear Regresson": LinearRegression(),
    "Ridge Regressor": Ridge(),
    "Lasso Regressor ": Lasso(),
    "Elastic Net Regressor": ElasticNet(),
    "SVR": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBRegressor": XGBRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
    }

In [27]:
# Create a function which can evaluate models and return a report 
def evaluate_models(X, y, models):
    '''
    This function takes in X and y and models dictionary as input
    It splits the data into Train Test split
    Iterates through the given model dictionary and evaluates the metrics
    Returns: Dataframe which contains report of all models metrics.
    '''
    # separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42)
    
    # Applying Satndard Normalization method (Z-Score Equation):
    scalar = StandardScaler()
    X_train = scalar.fit_transform(X_train)
    X_test = scalar.transform(X_test)
    
    models_list = []
    MAE_train_list = []
    MAE_test_list = []
    MSE_train_list = []
    MSE_test_list = []
    RMSE_train_list = []
    RMSE_test_list = []
    r2score_train_list = []
    r2score_test_list = []
    r2score_diff = []
    
    for i in range(len(list(models))):
        model = list(models.values())[i]
        wrapper = MultiOutputRegressor(model)
        wrapper.fit(X_train, y_train) # Train model

        # Make predictions
        y_train_pred = wrapper.predict(X_train)
        y_test_pred = wrapper.predict(X_test)

        # Training set performance
        model_train_MAE, model_train_MSE, model_train_RMSE,\
        model_train_r2score = evaluate_clf(y_train ,y_train_pred)
        
        MAE_train_list.append(model_train_MAE)
        MSE_train_list.append(model_train_MSE)
        RMSE_train_list.append(model_train_RMSE)
        r2score_train_list.append(model_train_r2score*100)
    
        # Test set performance
        model_test_MAE,model_test_MSE,model_test_RMSE,\
        model_test_r2score = evaluate_clf(y_test, y_test_pred)
        
        MAE_test_list.append(model_test_MAE)
        MSE_test_list.append(model_test_MSE)
        RMSE_test_list.append(model_test_RMSE)
        r2score_test_list.append(model_test_r2score*100)
        
        # r2score_train_test_difference
        r2score_diff.append(max(model_train_r2score, model_test_r2score) - 
                            min(model_train_r2score, model_test_r2score))  
    
        print(list(models.keys())[i])
        models_list.append(list(models.keys())[i])

        print('Model performance for Training set')
        print("- MAE: {:.4f}".format(model_train_MAE))
        print('- MSE: {:.4f}'.format(model_train_MSE)) 
        print('- RMSE: {:.4f}'.format(model_train_RMSE))
        print('- r2score: {:.4f}'.format(model_train_r2score))

        print('----------------------------------')

        print('Model performance for Test set')
        print('- MAE: {:.4f}'.format(model_test_MAE))
        print('- MSE: {:.4f}'.format(model_test_MSE))
        print('- RMSE: {:.4f}'.format(model_test_RMSE))
        print('- r2score: {:.4f}'.format(model_test_r2score))
        
        print('='*35)
        print('\n')    
        
    report = pd.DataFrame(list(zip(models_list, MAE_train_list, MAE_test_list, MSE_train_list, MSE_test_list, RMSE_train_list, RMSE_test_list, r2score_train_list,  r2score_test_list, r2score_diff)), columns=['Model Name', 'MAE_train', 'MAE_test', 'MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'r2score_train', 'r2score_test','r2score_diff']).sort_values(by='r2score_test',ascending=False)
        
    return report

In [28]:
report = evaluate_models(X, y, models)

Linear Regresson
Model performance for Training set
- MAE: 17.5807
- MSE: 1602.0695
- RMSE: 40.0259
- r2score: 0.0616
----------------------------------
Model performance for Test set
- MAE: 13.8839
- MSE: 600.1911
- RMSE: 24.4988
- r2score: 0.0912


Ridge Regressor
Model performance for Training set
- MAE: 17.5809
- MSE: 1602.0850
- RMSE: 40.0261
- r2score: 0.0616
----------------------------------
Model performance for Test set
- MAE: 13.9201
- MSE: 601.1638
- RMSE: 24.5186
- r2score: 0.0903


Lasso Regressor 
Model performance for Training set
- MAE: 17.5838
- MSE: 1602.9287
- RMSE: 40.0366
- r2score: 0.0296
----------------------------------
Model performance for Test set
- MAE: 14.1422
- MSE: 606.2384
- RMSE: 24.6219
- r2score: 0.0605


Elastic Net Regressor
Model performance for Training set
- MAE: 17.6292
- MSE: 1615.1141
- RMSE: 40.1885
- r2score: 0.0410
----------------------------------
Model performance for Test set
- MAE: 14.9514
- MSE: 634.5413
- RMSE: 25.1901
- r2score: 0

In [29]:
report

,Model Name,MAE_train,MAE_test,MSE_train,MSE_test,RMSE_train,RMSE_test,r2score_train,r2score_test,r2score_diff
10,Random Forest Regressor,8.275106,7.661275,341.147888,177.167466,18.470189,13.310427,79.688086,66.104386,0.135837
7,Gradient Boosting,4.490578,5.706396,56.465441,95.768346,7.514349,9.786130,96.971578,56.700040,0.402715
9,Decision Tree Regressor,0.343750,5.900000,2.446875,97.550000,1.564249,9.876740,99.320131,49.905195,0.494149
5,K-Neighbors Regressor,16.017500,11.600000,1348.780000,541.348000,36.725740,23.266886,20.043366,28.017093,0.079737
6,AdaBoost Regressor,13.268187,12.582419,799.151745,540.252479,28.269272,23.243332,68.912602,21.512117,0.474005
0,Linear Regresson,17.580735,13.883913,1602.069510,600.191075,40.025861,24.498797,6.159988,9.115915,0.029559
1,Ridge Regressor,17.580918,13.920061,1602.084956,601.163820,40.026053,24.518642,6.159049,9.033023,0.028740
2,Lasso Regressor,17.583754,14.142214,1602.928687,606.238364,40.036592,24.621908,2.959257,6.049291,0.030900
3,Elastic Net Regressor,17.629167,14.951446,1615.114092,634.541261,40.188482,25.190102,4.099269,5.072964,0.009737
8,XGBRegressor,0.592887,12.072101,2.635528,772.203181,1.623431,27.788544,99.314431,-4.268208,1.035826


In [30]:
final_model = RandomForestRegressor(n_estimators = 1000, max_depth = 5, random_state=42, n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42)

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)
wrapper = MultiOutputRegressor(final_model)

final_model = wrapper.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

print("Final RandomForest Regressor Accuracy Score (Train) :", final_model.score(X_train,y_train))
print("Final RandomForest Regressor Accuracy Score (Test) :", r2_score(y_pred,y_test))

Final RandomForest Regressor Accuracy Score (Train) : 0.72373720446705
Final RandomForest Regressor Accuracy Score (Test) : 0.1316418432486436


In [31]:
# Let's predict:
print(X_test[2])
y_pred = final_model.predict([X_test[0]])
y_pred
print("Nnumber of Likes: ", y_pred[0][0])
print("Time since posted: ", y_pred[0][1], 'hours')

[0.15081552]
Nnumber of Likes:  16.495048439146856
Time since posted:  2.1065460564241234 hours


In [36]:
X_train[3]

array([-0.71693828])

In [34]:
# Let's predict:
print(X_train[3])
y_pred = final_model.predict([X_train[0]])
y_pred
print("Nnumber of Likes: ", y_pred[0][0])
print("Time since posted: ", y_pred[0][1], 'hours')

[-0.71693828]
Nnumber of Likes:  219.62770225716233
Time since posted:  15.682177835762 hours
